In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import urllib.request
from bs4 import BeautifulSoup
import time
import random
import re

# The JASPAR website is saved locally and scraped for all human transcription factors to 
# mytuples.

f = open("../JASPAR_web.html")
alltext = f.read()
f.close()
mytuples = re.findall(
    r'>\s*(\w+\.\w+)\s*&nbsp;\s*</td>\s*\n.*\n'
    '.*<!-.+->\n'
    '.*<td.+>\s*(\w+.*)\s*&nbsp;\s*</td>\s*'
    '\n.*\n.*<!-.+->\s*\n.*'
    '<td.+Homo sapiens.+</td>',
    alltext, re.MULTILINE)

filenames = [thing[0] for thing in mytuples]

# Part 1
# This block generates files (scraped in Part 2) from the individual transcription factor pages

your_url_list = ["http://jaspar.genereg.net/cgi-bin/jaspar_db.pl?ID=" + filename +"&rm=present&collection=CORE" for filename in filenames]
                
file_name_list = filenames

for index, url in enumerate(your_url_list):
    try:
        page = urllib.request.urlopen(url)
        soup = BeautifulSoup(page)
        html = str(soup.find_all("html"))
        
        # Save the html to local hard drive first.
        file = open(file_name_list[index], "w")
        file.writelines(html)
        file.close()
        
        random_no = (random.random()-0.5) * 2 * 1
        time.sleep(2+random_no)
    except IndexError:
        pass
   


//anaconda/envs/my_projects_env/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file //anaconda/envs/my_projects_env/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [3]:
# Part 2
# This block creates a database (.txt file) of transcription factor binding sites by scraping
# the pages copied locally in Part 1.  The database is scanned against promoter sequences to 
# generate a Position Specific Motif Matrix (PSMM).  The database is structured according to 
# the format needed by CLOVER (used to generate the list of TF binding sites in each promoter).

import numpy as np, pandas as pd

open("outfile.txt", "w")
scrapped_data = []
for tup in mytuples:
    file = tup[0]
    genename = tup[1]
    soup = BeautifulSoup(open(file),"lxml")
    with open("outfile.txt", "a") as myfile:
        myfile.write(">"+file+" "+genename+"\n")
    mat = soup.find_all("pre")
    textmat = str(mat)
    textmat2 = re.findall(r'\[<pre>(.*)\n(.*)\n(.*)\n(.*\])', textmat, re.MULTILINE)[0]

    list_of_lists = []
    for i in list(range(4)):
        textmax3 = re.findall(r'\[(.*)\]',textmat2[i])[0].split(' ')
        itemlist=[]
        for item in textmax3:
            try:
                itemlist.append(int(item))
            except ValueError:
                pass
        list_of_lists.append(itemlist)
    finalmat = np.array(list_of_lists)
    finmat = finalmat.T
    df = pd.DataFrame(finmat)
    with open('outfile.txt', 'a') as f:
        df.to_csv(f, sep='\t', header=False, index=False)